# High-level RNN Keras (TF) Example

In [1]:
import os
import sys
import numpy as np
os.environ['KERAS_BACKEND'] = "tensorflow"
import keras as K
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, CuDNNGRU
from common.params_lstm import *
from common.utils import *

Using TensorFlow backend.
/opt/conda/envs/py3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Keras: ", K.__version__)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tf.__version__)
print(K.backend.backend())

OS:  linux
Python:  3.6.3 |Anaconda, Inc.| (default, Nov 20 2017, 20:41:42) 
[GCC 7.2.0]
Keras:  2.1.1
Numpy:  1.13.3
Tensorflow:  1.4.0
tensorflow


In [3]:
def create_symbol(CUDNN=True):
    model = Sequential()
    model.add(Embedding(MAXFEATURES, EMBEDSIZE, input_length=MAXLEN))
    # Only return last output
    if not CUDNN:
        model.add(GRU(NUMHIDDEN, return_sequences=False, return_state=False))
    else:
        model.add(CuDNNGRU(NUMHIDDEN, return_sequences=False, return_state=False))
    model.add(Dense(2, activation='softmax'))
    return model

In [4]:
def init_model(m):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = K.optimizers.Adam(LR, BETA_1, BETA_2, EPS),
        metrics = ['accuracy'])
    return m

In [5]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = imdb_for_library(seq_len=MAXLEN, max_features=MAXFEATURES, one_hot=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
Trimming to 30000 max-features
Padding to length 150
(25000, 150) (25000, 150) (25000, 2) (25000, 2)
int32 int32 int32 int32
CPU times: user 5.14 s, sys: 268 ms, total: 5.41 s
Wall time: 5.41 s


In [6]:
%%time
# Load symbol
sym = create_symbol()

CPU times: user 1.25 s, sys: 87.9 ms, total: 1.34 s
Wall time: 640 ms


In [7]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 256 ms, sys: 7.83 ms, total: 263 ms
Wall time: 44.2 ms


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 125)          3750000   
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (None, 100)               68100     
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 3,818,302
Trainable params: 3,818,302
Non-trainable params: 0
_________________________________________________________________


In [9]:
%%time
# 35s
# Train model
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

Epoch 1/3
25000/25000 [==============================] - 13s 515us/step - loss: 0.4762 - acc: 0.7525
Epoch 2/3
25000/25000 [==============================] - 11s 440us/step - loss: 0.2200 - acc: 0.9167
Epoch 3/3
25000/25000 [==============================] - 11s 441us/step - loss: 0.1094 - acc: 0.9641
CPU times: user 25.3 s, sys: 8.84 s, total: 34.1 s
Wall time: 35.5 s


In [10]:
%%time
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)

CPU times: user 2.33 s, sys: 881 ms, total: 3.21 s
Wall time: 2.97 s


In [11]:
print("Accuracy: ", sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.84984
